# Example 1 Simple equilibrium calculation with TC-Python

## Define problem
As a demonstration, we will find the equilibrium phases and the volume fraction of corresponding phases through TC-Python.
The system that we are interested in is the Fe-Cr-C system with 9wt%Cr and 0.2wt% C at 1773 K 1 bar.
We will use TCFE11 database to conduct this calcualtion.

## The general workflow to calculate equilibrium properties follows
1. Set a NTP condition (N: Element compositions, T: Temperature, P: Pressure),
2. Import databases,
3. Select phases (Recommended phase selection is given by default),
4. Calculate,
5. Data processsing.






## 1. Calculation Preparation
First we import the following libraries: numpy, pandas, and tc_python.
Step 1. we define the system and NTP condition in the python variables.
Note that this approach will make it easier to functionalize the code blocks for later application.

In [ ]:
import numpy as np, pandas as pd
from tc_python import *


database='TCFE11'           #Define database
elements=['Fe','Cr','C']    #Define system
compositions=[1,0.09,0.002] #Weight fraction for each elements. The first element value is dependent
temperature=1773            #Absolute temperature in Kelvin
pressure=1e5                #Pressure in pascal


## 2. TC-Python Calculation
We start the calculation by calling a session in tc_python.
Please make sure to close the session when calculations are completed.

In [ ]:

# Start TCPython session
TC = TCPython()
sess = TC.__enter__()

# Make sure you close the settion by
# TC.__exit__(sess,0,0)

# Alternatively, you can use 
# with TCPython() as sess:

Next, we define the system of interest, database, and phases.
This part completes the workflow of 2. Import database and 3. Phase selection.
We uses a calculation method called 'single equilibrium calculation' to find equilibrium.

In [ ]:

TCcalc = (
            sess
            .select_database_and_elements(database, elements)   #Import database & select elements
            .select_phase('FCC_L12')                            #select a phase
            #.deselect_phase('BCC_B2')                          #deselect a phase
            .get_system()                                       #Complete the phase selection
            .with_single_equilibrium_calculation()              #Define calculation method
            .enable_global_minimization()                       #Enable global minimization
        )



Now we provide the condition to find the phase equilibria.
Note that TC-Python takes the first element of the list as a dependent element when we set the composition.
After setting the condition, conduct the calculation and save the results in 'result' variable.

In [ ]:

#When the composition is given in wt fraction, take a function from tc_python that defines it.
wf = ThermodynamicQuantity.mass_fraction_of_a_component
for i in range(1,len(elements)):
    TCcalc.set_condition(wf(elements[i]), compositions[i])
TCcalc.set_condition(ThermodynamicQuantity.temperature(), temperature)
TCcalc.set_condition(ThermodynamicQuantity.pressure(), pressure)
result = TCcalc.calculate()

The variable 'result' contains many classes of variables. 
We need to process the data to obtain the names of stable phases and corresponding volume fractions.

In [ ]:
phases=result.get_stable_phases()
output =dict({})
for phase in phases:
    VPV=result.get_value_of("VPV({})".format(phase))
    output['VPV_{}'.format(phase)]=VPV

# Close TCPython session
TC.__exit__(sess,0,0)

print(output)